# View Campaign and Interactions

In the first notebook `Personalize_BuildCampaign.ipynb` you successfully built and deployed a recommendation model using deep learning with Amazon Personalize.

This notebook will expand on that and will walk you through adding the ability to react to real time behavior of users. If their intent changes while browsing a movie, you will see revised recommendations based on that behavior.

It will also showcase demo code for simulating user behavior selecting movies before the recommendations are returned.

Below we start with just importing libraries that we need to interact with Personalize

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

The line below will retrieve your shared variables from the first notebook.

In [2]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')
%store -r dataset_group_arn
%store -r PostClickEvent_arn
%store -r bucket

no stored variable bucket


## Creating an Event Tracker

Before your recommendation system can respond to real time events you will need an event tracker, the code below will generate one and can be used going forward with this lab. Feel free to name it something more clever.

In [3]:
response = personalize.create_event_tracker(
    name='AllstoreClickTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

arn:aws:personalize:us-east-1:962222257213:event-tracker/9c23a5a5
bfff7247-1fb1-46ca-b80d-8abc1bf675f6


## Please note one Dataset Group can only have one "event tracker".

In [4]:
event_tracker_arn = response['eventTrackerArn']

## Configuring Source Data

Above you'll see your tracking ID and this has been assigned to a variable so no further action is needed by you. The lines below are going to setup the data used for recommendations so you can render the list of Allstore products later.

In [7]:
%store -r bucket
# First load items into memory
allitemuri = f's3://{bucket}/items_w_metadata.csv'
items = pd.read_csv(allitemuri,sep=',', usecols=[0,1], encoding='utf-8', names=['ITEM_ID', 'TITLE','IMGURL','GENRE'], index_col='ITEM_ID')

def get_product_title(product_id):
    """
    Takes in an ID, returns a title
    """
    product_id = str(product_id)
    return items.loc[product_id]['TITLE']

## Getting Recommendations

First we will render the recommendations again from the previous notebook 1

In [31]:
%store -r recommendations_df
%store -r user_id
recommendations_df

,Item Description
0,Emergency Mylar Thermal Blankets (Pack of 10)
1,Park Tool Vulcanizing Patch Kit - VP-1
2,Potable Aqua Water Treatment Tablets
3,Petzl E91 Tikkina 2 Headlamp
4,GSI Outdoors Glacier Stainless Bottle Cup/Pot
5,Gerber 31-000699 Bear Grylls Survival Series F...
6,ALPS Mountaineering Compression Sleeping Bag S...
7,Eagles Nest Outfitters DryFly Raintarp
8,Paracord Planet Type III 7 Strand 550 Paracord...
9,Magnesium Fire Starter


## Simulating User Behavior

The lines below provide a code sample that simulates a user interacting with a particular item, you will then get recommendations that differ from those when you started.

In [9]:
session_dict = {}

In [10]:
def send_product_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
    
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
         'sentAt': int(time.time()),
         'eventType': 'EVENT_TYPE',
         'properties': event_json
         }]
)

Immediately below this line will update the tracker as if the user has clicked a particular title.


If the table generated by the cells below does not shift the recommendations simply try another random 3 digit number in the cell above and run both cells again. You'll see a third column generated of recommendations.

In [36]:
# Pick a product_ID, we will use '1881509818'
product_to_click = 1881509818
product_title_clicked = get_product_title(product_to_click)
send_product_click(USER_ID=str(user_id), ITEM_ID=product_to_click)

After executing this block you will see the alterations in the recommendations now that you have event tracking enabled and that you have sent the events to the service.

In [37]:
%store -r hrnn_campaign_arn
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = hrnn_campaign_arn,
    userId = str(user_id),
)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []
for item in item_list:
    title = get_product_title(item['itemId'])
    recommendation_list.append(title)
    
new_rec_DF = pd.DataFrame(recommendation_list,columns =['New Recommendations'])
com_recommendations_df = recommendations_df.join(new_rec_DF)
com_recommendations_df



Recommendations for user:  A2AM9IPXQKNO8L


,Item Description,New Recommendations
0,Emergency Mylar Thermal Blankets (Pack of 10),Emergency Mylar Thermal Blankets (Pack of 10)
1,Park Tool Vulcanizing Patch Kit - VP-1,Rothco 550lb. Type III Nylon Paracord
2,Potable Aqua Water Treatment Tablets,Butler Creek Lula Universal Pistol Magazine Lo...
3,Petzl E91 Tikkina 2 Headlamp,ALPS Mountaineering Compression Sleeping Bag S...
4,GSI Outdoors Glacier Stainless Bottle Cup/Pot,Condor Molle Gadget Pouch
5,Gerber 31-000699 Bear Grylls Survival Series F...,Petzl E91 Tikkina 2 Headlamp
6,ALPS Mountaineering Compression Sleeping Bag S...,Tac Force TF-705 Series Assisted Opening Foldi...
7,Eagles Nest Outfitters DryFly Raintarp,Morakniv Companion Fixed Blade Outdoor Knife w...
8,Paracord Planet Type III 7 Strand 550 Paracord...,WarTech USA Batman Knife with Dual Assist Open...
9,Magnesium Fire Starter,Champion Men's Compression Short


## Update lambda environment var

In [38]:
lambdaclient = boto3.client('lambda')

response = lambdaclient.get_function(
    FunctionName=PostClickEvent_arn,
)

Environment = response['Configuration']['Environment']
print(Environment)

#update Env var

Environment['Variables']['TRACKING_ID'] = TRACKING_ID

response = lambdaclient.update_function_configuration(
    FunctionName=PostClickEvent_arn,
    Environment=Environment
)

{'Variables': {'TRACKING_ID': '8b1ce80f-3c86-4924-aaeb-f4a7bb992a53'}}
